In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
import seaborn as sns
import pandas as pd

In [ ]:
def add_noise(image, model, mask=None):
    image = np.array(image, copy=False)
    shape = image.shape

    noisy = np.array(list(map(model, image.ravel())), dtype=image.dtype)
    noisy.shape = shape

    if mask is not None:
        mask = np.logical_not(np.logical_and(np.isfinite(image), mask))
        noisy[mask] = image[mask]

    return noisy

def filter(image, sigma):
    if sigma == 0:
        return image
    return scipy.ndimage.filters.gaussian_filter(image, sigma)

def resample(image, scale):
    sx = int(round(image.shape[0] * scale))
    sy = int(round(image.shape[1] * scale))
    data = np.zeros((sx, sy), dtype=image.dtype)
    divisor = np.zeros((sx, sy), dtype=np.int64)

    for i in range(image.shape[0]):
        ii = int(np.floor((i + 0.5) * scale))
        if ii >= data.shape[0]:
            continue
        for j in range(image.shape[1]):
            jj = int(np.floor((j + 0.5) * scale))
            if jj >= data.shape[1]:
                continue
            divisor[ii, jj] += 1
            data[ii, jj] += image[i, j]
    data = np.divide(data, divisor)

    return data

def filter_resample(image, sigma, scale):
    return resample(filter(image, sigma), scale)



In [ ]:
input = np.zeros((1000,1000))


In [ ]:


scale = voxel_size / pixel_size
size = np.array(np.round(np.array(input.shape[0:2]) * scale), dtype=int)

In [ ]:
def rect(x, d):
    r = np.zeros_like(x)
    r[np.bitwise_and(np.abs(x) > 0.5 * d, np.abs(x) < 1.5 * d)] = 1
    return r

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def discret(x, data, delta, xstart):
    # h, hx = np.histogram(data, int((np.max(data) + np.min(data))/delta))
    # x = x[:-1] + (x[1]-x[0])/2

    r = np.arange(xstart, np.max(x)+delta, delta)

    h = []
    for r0, r1 in zip(r[:-1], r[1:]):
        sub = data[np.logical_and(x>=r0, x< r1)]
        h.append(np.mean(sub))

    hx = r[:-1] + delta/2
    
    return np.array(h),hx

''' resolution ~ distance between two ringing function, when the sum is 1/2.
    ~ 1/2 * line width of USAF when min(signal) ~ 1/2 max(signal)
'''



d = 49.61
p = 20
g = 1.2*p

x = np.linspace(-4 * d, 4 * d, 1000)
X = rect(x, d)
Y = gaussian(x, 0, g)
Z = np.convolve(X, Y, 'same')


X /= np.sum(X*(x[1]-x[0]))
Y /= np.sum(Y*(x[1]-x[0]))
Z /= np.sum(Z*(x[1]-x[0]))
plt.plot(x, X)
# plt.plot(x, Y)
plt.plot(x, Z)

for d in np.linspace(0, d, 5):
    D, xd = discret(x, Z, p, np.min(x)+d)

    if len(xd)>1:
        D /= np.sum(D*(xd[1]-xd[0]))

        delta_x = xd[1]-xd[0]
        for i, (x, d) in enumerate(zip(xd, D)):
            if i == 0:
                plt.plot([x-delta_x/2,x-delta_x/2,x+delta_x/2],[0,d,d], 'k')
            else:
                plt.plot([x-delta_x/2,x-delta_x/2,x+delta_x/2],[d_last,d,d], 'k')
            d_last = d

    # if len(xd) > 1:
    #     plt.plot(xd, D)
    plt.scatter(xd, D, c='k')
plt.show()
